## Shell Cantilever Example

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [2]:
import os
import cProfile
os.environ["MKL_THREADING_LAYER"] = "INTEL" # for pardiso: if the kernel crashes, try changing "GNU" to "INTEL"

In [3]:
import numpy as np

import dismech

geom = dismech.GeomParams(rod_r0=0,
                          shell_h=1e-3)

material = dismech.Material(density=1200,
                            youngs_rod=0,
                            youngs_shell=2e8,
                            poisson_rod=0,
                            poisson_shell=0.5)

dynamic_3d_sim = dismech.SimParams(static_sim=False,
                                   two_d_sim=False,
                                   use_mid_edge=True,
                                   use_line_search=False,
                                   show_floor=False,
                                   log_data=True,
                                   log_step=1,
                                   dt=1e-2,
                                   max_iter=25,
                                   total_time=1.0,
                                   plot_step=10,
                                   tol=1e-4,
                                   ftol=1e-4,
                                   dtol=1e-2,
                                   sparse=True,
                                   solver='pardiso')

env = dismech.Environment()
env.add_force('gravity', g=np.array([0.0, 0.0, -9.81]))

geo = dismech.Geometry.from_txt(
    '../tests/resources/shell_cantilever/equilateral_mesh_40.txt')

robot = dismech.SoftRobot(geom, material, geo, dynamic_3d_sim, env)

### Time Stepping

We only need to fix the first few nodes

In [4]:
fixed_points = np.array(np.where(robot.state.q[:robot.end_node_dof_index].reshape(-1, 3)[:robot.end_node_dof_index, 0] <= 0.01)[0])
robot = robot.fix_nodes(fixed_points)

stepper = dismech.NewmarkBetaTimeStepper(robot)

In [5]:
cProfile.run('stepper.simulate()', sort='cumtime')

  0%|          | 0/100 [00:00<?, ?it/s]

         1838653 function calls (1835142 primitive calls) in 26.188 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  320/319    0.005    0.000   45.721    0.143 base_events.py:1894(_run_once)
      100    0.179    0.002   25.260    0.253 time_stepper.py:63(step)
      461    0.572    0.001   17.017    0.037 time_stepper.py:160(_compute_forces_and_jacobian)
      461    6.748    0.015   13.481    0.029 triangle_energy.py:99(grad_hess_energy_linear_elastic)
      461    0.003    0.000    7.282    0.016 solver.py:35(solve)
      461    0.007    0.000    7.279    0.016 scipy_aliases.py:12(spsolve)
      922    6.861    0.007    6.938    0.008 pardiso_wrapper.py:266(_call_pardiso)
      461    0.012    0.000    6.605    0.014 pardiso_wrapper.py:145(factorize)
    13869    0.013    0.000    2.745    0.000 einsumfunc.py:1057(einsum)
    13869    2.731    0.000    2.731    0.000 {built-in method numpy._core._multiarray_umath.c_e

In [6]:
%lprun -f stepper._TimeStepper__elastic_energies[1].grad_hess_energy_linear_elastic stepper.simulate()

  0%|          | 0/100 [00:00<?, ?it/s]

Timer unit: 1e-07 s

Total time: 14.1813 s
File: C:\Users\12345\Documents\Dev\dismech-python\src\dismech\elastics\triangle_energy.py
Function: grad_hess_energy_linear_elastic at line 99

Line #      Hits         Time  Per Hit   % Time  Line Contents
    99                                               def grad_hess_energy_linear_elastic(self, state: RobotState, sparse=False) -> typing.Tuple[np.ndarray, np.ndarray]:
   100       465     477202.0   1026.2      0.3          tau = self._get_tau(state.tau)
   101       465      48897.0    105.2      0.0          xis = self._get_xi_is(state.q)
   102       465    6776699.0  14573.5      4.8          t, f, c, unit_norm = self._get_t_f_c(state.q, tau)
   103       465       6469.0     13.9      0.0          N = t.shape[0]
   104                                           
   105                                                   # s_s terms
   106       465      59749.0    128.5      0.0          s_xis = self._s_s * xis - f
   107               